# Goals

1. prototype some pangeoLib functions
2. organize notebooks for inputs and output classification for ET

## prototype
1. boto 3 list all tiles
2. boto 3 list all outputs in tiles by subset
3. build coverage geometries for each tile day 001

In [ ]:
import boto3

In [ ]:
from pangeoLib.aws_authenticate import aws_authenticate

aws_authenticate()

In [ ]:
#list_all_tiles()

In [ ]:
# ! aws s3 ls --request-payer requester dev-et-data/tiles/

# rio takes too long - research ways to just read the header

## USGS network security should be investigated again for slow speed ???

In [ ]:
! rio info /vsis3/dev-et-data/test/compressed/NDVI_filled/2001/2001001.250_m_NDVI.tif

In [ ]:
import fsspec
fs = fsspec.filesystem('s3', anon=False, requester_pays=True)
chip_list = fs.ls('dev-et-data/tiles/')

In [ ]:
! rio info /vsis3/dev-et-data/tiles/tile40N-100E_chip32N-100E/dd_2014.tif

In [ ]:
import rasterio
file='/vsis3/dev-et-data/tiles/tile40N-100E_chip32N-100E/dd_2014.tif'
src = rasterio.open(file)
src.shape

In [ ]:
#dir(src)

src.bounds.left

In [ ]:
#dir(src.bounds)



You are almost there with what you tried, and using the box method is indeed the best way. With the list comprehension, you can do

b = [box(l, b, r, t) for l, b, r, t in zip(df.left, df.bottom, df.right, df.top)]

Another option is to apply the box function to each row of your dataframe:

b = df.apply(lambda row: box(row.left, row.bottom, row.right, row.top), axis=1)

Once you have converted the bounding boxes to polygons, make sure to actually create a GeoDataFrame:

gdf = geopandas.GeoDataFrame(df, geometry=b)



In [ ]:
# chip_list

In [ ]:
def get_bounding(file):
    try:
        src = rasterio.open(file)
        return(src.bounds)
    except:
        pass
        

In [ ]:

import pandas as pd

def build_df_of_chip_bounds():
    df = pd.DataFrame(columns=('Chip', 'left', 'bottom', 'right', 'top'))
    i=0
    for chip in chip_list:
        file = '/vsis3/' + chip + '/dd_2013.tif'
        bounds = get_bounding(file)
        if not bounds is None:
            print(bounds.left)
            df.loc[i] =[chip, bounds.left, bounds.bottom, bounds.right, bounds.top]
            i=i+1
    return(df)

In [ ]:
import time
try:
    df = pd.read_pickle('tile_catalog_df.pkl')
except:
    start = time.time()
    print("build df one time")
    df = build_df_of_chip_bounds()
    end = time.time()
    print("Time it Took =",end - start)




In [ ]:
df.head(30)

In [ ]:
#dir(df)

In [ ]:
df.describe()

In [ ]:
import geopandas
from shapely.geometry import box
#import shapely.bounding.box as box

#Once you have converted the bounding boxes to polygons, make sure to actually create a GeoDataFrame:
b = df.apply(lambda row: box(row.left, row.bottom, row.right, row.top), axis=1)
gdf = geopandas.GeoDataFrame(df, geometry=b)

In [ ]:
gdf.head()

In [ ]:
import folium


In [ ]:
tony_plot_tiles_json = gdf.to_json()

In [ ]:

map_osm = folium.Map(location=[35, -90], zoom_start=5)

tile_boxes=folium.features.GeoJson(tony_plot_tiles_json)
map_osm.add_child(tile_boxes)
map_osm

In [ ]:
my_s= gdf.loc[0]

#print(dir(my_s))


print(my_s.geometry)



In [ ]:
geopandas.GeoSeries([my_s.geometry]).to_json()

In [ ]:
map_osm = folium.Map(location=[35, -90], zoom_start=5)

style_function = lambda x: {'fillColor': 'red', 'color': 'green'} 

for index, row in gdf.iterrows():
    # print(row.geometry)
    # print(row.Chip)
    my_useful_geojson = geopandas.GeoSeries([row.geometry]).to_json()
    gjson = folium.GeoJson(my_useful_geojson, style_function=style_function, tooltip=row.Chip).add_to(map_osm)



map_osm

In [ ]:
df.to_pickle('tile_catalog_df.pkl')  # where to save it, usually as a .pkl


df = pd.read_pickle('tile_catalog_df.pkl')

In [ ]:
df.head()